In [23]:
import json
import base64
import qrcode
import requests
import datetime
import urllib.parse
import uuid
import matplotlib.pyplot as plt
from pymongo import MongoClient
from typing import Optional, List
from didcomm.common.types import DID, VerificationMethodType, VerificationMaterial, VerificationMaterialFormat
from didcomm.did_doc.did_doc import DIDDoc, VerificationMethod, DIDCommService
from didcomm.did_doc.did_resolver import DIDResolver
from didcomm.message import Message, FromPrior
from didcomm.secrets.secrets_resolver_demo import SecretsResolverDemo
from didcomm.unpack import unpack, UnpackResult
from didcomm.common.resolvers import ResolversConfig
from didcomm.pack_encrypted import pack_encrypted, PackEncryptedConfig, PackEncryptedResult
from peerdid.core.did_doc_types import DIDCommServicePeerDID
from didcomm.secrets.secrets_util import generate_x25519_keys_as_jwk_dict, generate_ed25519_keys_as_jwk_dict, jwk_to_secret
from peerdid import peer_did
from peerdid.did_doc import DIDDocPeerDID
from didcomm.message import Attachment, AttachmentDataJson
from peerdid.types import VerificationMaterialAuthentication, VerificationMethodTypeAuthentication, VerificationMaterialAgreement, VerificationMethodTypeAgreement, VerificationMaterialFormatPeerDID

In [24]:
secrets_resolver = SecretsResolverDemo()

In [25]:
class DIDResolverPeerDID(DIDResolver):
    async def resolve(self, did: DID) -> DIDDoc:
        did_doc_json = peer_did.resolve_peer_did(did, format = VerificationMaterialFormatPeerDID.JWK)
        did_doc = DIDDocPeerDID.from_json(did_doc_json)

        return DIDDoc(
            did=did_doc.did,
            key_agreement_kids = did_doc.agreement_kids,
            authentication_kids = did_doc.auth_kids,
            verification_methods = [
                VerificationMethod(
                    id = m.id,
                    type = VerificationMethodType.JSON_WEB_KEY_2020,
                    controller = m.controller,
                    verification_material = VerificationMaterial(
                        format = VerificationMaterialFormat.JWK,
                        value = json.dumps(m.ver_material.value)
                    )
                )
                for m in did_doc.authentication + did_doc.key_agreement
            ],
            didcomm_services = [
                DIDCommService(
                    id = s.id,
                    service_endpoint = s.service_endpoint,
                    routing_keys = s.routing_keys,
                    accept = s.accept
                )
                for s in did_doc.service
                if isinstance(s, DIDCommServicePeerDID)
            ] if did_doc.service else []
        )

In [26]:
async def create_peer_did(self,
                        auth_keys_count: int = 1,
                        agreement_keys_count: int = 1,
                        service_endpoint: Optional[str] = None,
                        service_routing_keys: Optional[List[str]] = None
                        ) -> str:
        # 1. generate keys in JWK format
        agreem_keys = [generate_x25519_keys_as_jwk_dict() for _ in range(agreement_keys_count)]
        auth_keys = [generate_ed25519_keys_as_jwk_dict() for _ in range(auth_keys_count)]

        # 2. prepare the keys for peer DID lib
        agreem_keys_peer_did = [
            VerificationMaterialAgreement(
                type=VerificationMethodTypeAgreement.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in agreem_keys
        ]
        auth_keys_peer_did = [
            VerificationMaterialAuthentication(
                type=VerificationMethodTypeAuthentication.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in auth_keys
        ]

        # 3. generate service
        service = None
        if service_endpoint:
            service = json.dumps(
                DIDCommServicePeerDID(
                    id="new-id",
                    service_endpoint=service_endpoint, routing_keys=service_routing_keys,
                    accept=["didcomm/v2"]
                ).to_dict()
            )

        # 4. call peer DID lib
        # if we have just one key (auth), then use numalg0 algorithm
        # otherwise use numalg2 algorithm
        if len(auth_keys_peer_did) == 1 and not agreem_keys_peer_did and not service:
            did = peer_did.create_peer_did_numalgo_0(auth_keys_peer_did[0])
        else:
            did = peer_did.create_peer_did_numalgo_2(
                encryption_keys=agreem_keys_peer_did,
                signing_keys=auth_keys_peer_did,
                service=service,
            )

        # 5. set KIDs as in DID DOC for secrets and store the secret in the secrets resolver
        did_doc = DIDDocPeerDID.from_json(peer_did.resolve_peer_did(did))
        for auth_key, kid in zip(auth_keys, did_doc.auth_kids):
            private_key = auth_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        for agreem_key, kid in zip(agreem_keys, did_doc.agreement_kids):
            private_key = agreem_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        return did


### Receiving the OOB message

In [27]:
#oob_url = requests.get("https://mediator.rootsid.cloud/oob_url").text
oob_url = requests.get("http://127.0.0.1:8000/oob_url").text
print(oob_url)

http://127.0.0.1:8000?_oob=eyJ0eXBlIjoiaHR0cHM6Ly9kaWRjb21tLm9yZy9vdXQtb2YtYmFuZC8yLjAvaW52aXRhdGlvbiIsImlkIjoiODE3MDAwZTEtYjczYi00ZjY5LWFmMjAtNDI0NGU0MWQ2NjEzIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNjeHQzdGZkUHB5NEN2c0pZSFk5WmVZaGdFRzkxVGZCMW15SlAzQmQ3TThKWS5WejZNa2tVM1hNREZHa0RMSEJ6NlZkM2Vmakp3RURzQ1NHeWNjckJxTUJEQzNuR0JvLlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd09pOHZNVEkzTGpBdU1DNHhPamd3TURBaUxDSmhJanBiSW1ScFpHTnZiVzB2ZGpJaVhYMCIsImJvZHkiOnsiZ29hbF9jb2RlIjoicmVnaXN0ZXIiLCJnb2FsIjoiUmVnaXN0ZXJpbm1lZGlhdG9yIiwiYWNjZXB0IjpbImRpZGNvbW0vdjIiLCJkaWRjb21tL2FpcDI7ZW52PXJmYzU4NyJdfX0


In [28]:
received_msg_encoded = oob_url.split("=")[1]
received_msg_decoded = json.loads(str(base64.urlsafe_b64decode(received_msg_encoded + "=="), "utf-8"))
print(received_msg_decoded)

{'type': 'https://didcomm.org/out-of-band/2.0/invitation', 'id': '817000e1-b73b-4f69-af20-4244e41d6613', 'from': 'did:peer:2.Ez6LScxt3tfdPpy4CvsJYHY9ZeYhgEG91TfB1myJP3Bd7M8JY.Vz6MkkU3XMDFGkDLHBz6Vd3efjJwEDsCSGyccrBqMBDC3nGBo.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'body': {'goal_code': 'register', 'goal': 'Registerinmediator', 'accept': ['didcomm/v2', 'didcomm/aip2;env=rfc587']}}


## Prepare request-credential

Create Perr DID to communicate to issuer:

In [29]:
holder_did = await create_peer_did(1,1, service_endpoint="https://www.example.com/holder")
print("Holder's DID:", holder_did)

Holder's DID: did:peer:2.Ez6LScfqXt6TRDoQEoKV9P7MDowqT6hMzX3BfRKr5drVhvBa9.Vz6Mkmi3mGZaD9ai9RHRAMokKiCibAx7FTqWMDFfvPg2o1qdS.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL3d3dy5leGFtcGxlLmNvbS9ob2xkZXIiLCJhIjpbImRpZGNvbW0vdjIiXX0


Holders Prism DID:

In [30]:
holder_prism_did = "did:prism:a08f49e61bc46dc82b6bd54dd62087c25b53b4a7ef98e549ce62ee4ad3450d5c"

Credential request:

In [31]:
credential_request = {
    "credential": {
        "@context": 
        [
            "https://www.w3.org/2018/credentials/v1",
            "https://www.w3.org/2018/credentials/examples/v1"
        ],
        "id": str(uuid.uuid4()),
        "type": ["VerifiableCredential", "UniversityDegreeCredential"],
        "issuer": "TBD",
        "issuanceDate": datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"),
        "credentialSubject": 
            {
              "id": holder_prism_did,
              "degree": 
                {
                    "type": "BachelorDegree",
                    "name": "Bachelor of Science and Arts"
                  }
            },
        "options": {
            "proofType": "EcdsaSecp256k1Signature2019"
        }
    }
}

Holder create an request credendial message

In [40]:

holder_response_message = Message(
    custom_headers = [{
        "return_route": "all"}],
    id = str(uuid.uuid4()),
    #pthid = received_msg_decoded["id"],
    type = "https://didcomm.org/issue-credential/3.0/request-credential",
    frm = holder_did,
    to = [received_msg_decoded["from"]],
    body = {
        "goal_code": "issue-credential",
        "comment": "some comment"
    },
    attachments = [
        Attachment(
                id=str(uuid.uuid4()),
                media_type= "application/json",
                format= "aries/ld-proof-vc-detail@v1.0",
                data=AttachmentDataJson(json=credential_request)
                )
    ]
                        
)

In [41]:
holder_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = holder_response_message,
    frm = holder_did,
    to = received_msg_decoded["from"],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

### Sending the message to Mediator

In [42]:
issuer_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))
print(issuer_did_doc)

{'id': 'did:peer:2.Ez6LScxt3tfdPpy4CvsJYHY9ZeYhgEG91TfB1myJP3Bd7M8JY.Vz6MkkU3XMDFGkDLHBz6Vd3efjJwEDsCSGyccrBqMBDC3nGBo.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'authentication': [{'id': 'did:peer:2.Ez6LScxt3tfdPpy4CvsJYHY9ZeYhgEG91TfB1myJP3Bd7M8JY.Vz6MkkU3XMDFGkDLHBz6Vd3efjJwEDsCSGyccrBqMBDC3nGBo.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0#6MkkU3XMDFGkDLHBz6Vd3efjJwEDsCSGyccrBqMBDC3nGBo', 'type': 'Ed25519VerificationKey2020', 'controller': 'did:peer:2.Ez6LScxt3tfdPpy4CvsJYHY9ZeYhgEG91TfB1myJP3Bd7M8JY.Vz6MkkU3XMDFGkDLHBz6Vd3efjJwEDsCSGyccrBqMBDC3nGBo.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'publicKeyMultibase': 'z6MkkU3XMDFGkDLHBz6Vd3efjJwEDsCSGyccrBqMBDC3nGBo'}], 'keyAgreement': [{'id': 'did:peer:2.Ez6LScxt3tfdPpy4CvsJYHY9ZeYhgEG91TfB1myJP3Bd7M8JY.Vz6MkkU3XMDFGkDLHBz6Vd3efjJwEDsCSGyccrBqMBDC3nGBo.SeyJpZCI6Im5ldy1pZCIsI

Get Issuer end point:

In [43]:
issuer_endpoint = issuer_did_doc["service"][0]["serviceEndpoint"]
print(issuer_endpoint)

http://127.0.0.1:8000


Send message:

In [65]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(issuer_endpoint, headers=headers, data = holder_packed_msg.packed_msg)
print(resp.json())


{"protected":"eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC0xUFUrQTI1NktXIiwiZW5jIjoiQTI1NkNCQy1IUzUxMiIsImFwdSI6IlpHbGtPbkJsWlhJNk1pNUZlalpNVTJOaFUwTlpiblJNT1hSMlMyUkVZbWQyVVhKWWNXcGlhamxZTjJKYWFVeGtabUpIUVVwT1VtRk9aM2wwTGxaNk5rMXJaMVZ6TVZWeFkwdDRPVFo0Y0ZGWU16TnZibXA1UlZsRllXUndibEZ0U3pKR1JIbDRXV2RSVUU1bWFFNHVVMlY1U25CYVEwazJTVzAxYkdSNU1YQmFRMGx6U1c1UmFVOXBTbXRpVTBselNXNU5hVTlwU205a1NGSjNUMms0ZGsxVVNUTk1ha0YxVFVNMGVFOXFaM2ROUkVGcFRFTkthRWxxY0dKSmJWSndXa2RPZG1KWE1IWmtha2xwV0Znd0l6Wk1VMk5oVTBOWmJuUk1PWFIyUzJSRVltZDJVWEpZY1dwaWFqbFlOMkphYVV4a1ptSkhRVXBPVW1GT1ozbDAiLCJhcHYiOiJJWHNFc3p0enpHMmhxeFl2Y18yeW8taV91VGdVRHJxUmdXVHhIM0t1TVZBIiwic2tpZCI6ImRpZDpwZWVyOjIuRXo2TFNjYVNDWW50TDl0dktkRGJndlFyWHFqYmo5WDdiWmlMZGZiR0FKTlJhTmd5dC5WejZNa2dVczFVcWNLeDk2eHBRWDMzb25qeUVZRWFkcG5RbUsyRkR5eFlnUVBOZmhOLlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd09pOHZNVEkzTGpBdU1DNHhPamd3TURBaUxDSmhJanBiSW1ScFpHTnZiVzB2ZGpJaVhYMCM2TFNjYVNDWW50TDl0dktkRGJndlFyWHFqYmo5WDdiWm

### Credential Received

In [66]:
credential_unpack_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)

In [67]:
print(credential_unpack_msg)

UnpackResult(message=Message(id='07facd81-cf19-44cf-b848-885334d82829', type='https://didcomm.org/issue-credential/3.0/issue-credential', body={'goal_code': 'issue-credential', 'comment': 'some comment'}, frm=None, to=None, created_time=None, expires_time=None, from_prior=None, please_ack=None, ack=None, thid=None, pthid=None, attachments=[Attachment(data=AttachmentDataJson(json={'@context': ['https://www.w3.org/2018/credentials/v1', 'https://www.w3.org/2018/credentials/examples/v1'], 'id': 'a928d1e6-5c1d-4271-8a49-5f0241a39572', 'type': ['VerifiableCredential', 'UniversityDegreeCredential'], 'issuer': 'did:prism:712f7658d27166dc1f99d70cd24253eac553d5d16e8ae38b2972cafcbb21ca00', 'issuanceDate': '2022-08-30T15:58:22Z', 'credentialSubject': {'id': 'did:prism:a08f49e61bc46dc82b6bd54dd62087c25b53b4a7ef98e549ce62ee4ad3450d5c', 'degree': {'type': 'BachelorDegree', 'name': 'Bachelor of Science and Arts'}}, 'options': {'proofType': 'EcdsaSecp256k1Signature2019'}, 'proof': {'type': 'EcdsaSecp25

In [68]:
print(credential_unpack_msg.message.attachments[0].data.json)

{'@context': ['https://www.w3.org/2018/credentials/v1', 'https://www.w3.org/2018/credentials/examples/v1'], 'id': 'a928d1e6-5c1d-4271-8a49-5f0241a39572', 'type': ['VerifiableCredential', 'UniversityDegreeCredential'], 'issuer': 'did:prism:712f7658d27166dc1f99d70cd24253eac553d5d16e8ae38b2972cafcbb21ca00', 'issuanceDate': '2022-08-30T15:58:22Z', 'credentialSubject': {'id': 'did:prism:a08f49e61bc46dc82b6bd54dd62087c25b53b4a7ef98e549ce62ee4ad3450d5c', 'degree': {'type': 'BachelorDegree', 'name': 'Bachelor of Science and Arts'}}, 'options': {'proofType': 'EcdsaSecp256k1Signature2019'}, 'proof': {'type': 'EcdsaSecp256k1Signature2019', 'created': '2022-08-30T15:58:26Z', 'verificationMethod': 'did:prism:712f7658d27166dc1f99d70cd24253eac553d5d16e8ae38b2972cafcbb21ca00', 'proofPurpose': 'assertionMethod', 'proofValue': 'eyJpZCI6ImRpZDpwcmlzbTo3MTJmNzY1OGQyNzE2NmRjMWY5OWQ3MGNkMjQyNTNlYWM1NTNkNWQxNmU4YWUzOGIyOTcyY2FmY2JiMjFjYTAwIiwia2V5SWQiOiJpc3N1aW5nMCIsImNyZWRlbnRpYWxTdWJqZWN0Ijp7Imlzc3VhbmNlRG